<a href="https://colab.research.google.com/github/mssomie/Deep-Neural-Network/blob/main/Concrete_Strength_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building an Artificial Neural Network from Scratch


In [10]:
import random
import numpy as np

# Define activation functions

Logistics Function

In [11]:
def logisitics_function (x):
  return 1/(1+np.exp(-x))


RELU Function


In [35]:
def RELU(x):
  return np.maximum(0,x)


Hyperbolic Function

In [13]:
def hyberbolic_tangent(x):
    function = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    derivative = 1 - function ** 2
    return function, derivative

In [14]:
print(logisitics_function(10))
print(RELU(10))
print(hyberbolic_tangent(10))

0.9999546021312976
10
(0.9999999958776926, 8.244614768671e-09)


# Network


In [40]:
# TODO: fix for multiple hidden layers
class Multilayer_perceptron:

   # Initialize ANN
   def __init__(self, hidden_layers_neurons, activation_function, input_layer_neurons=8,  output_layer_neuron=1, bias_input =1):
       self.input_layer_neurons = input_layer_neurons
       self.hidden_layers_neurons = hidden_layers_neurons
       self.output_layer_neuron = output_layer_neuron
       self.weight = []
       self.bias = []   

       self.activation_function = activation_function

       # Input layer to hidden layer
       self.weight.append(np.random.rand(hidden_layers_neurons[0],input_layer_neurons))
       self.bias.append(np.random.rand(hidden_layers_neurons[0],bias_input))

      # Hidden layer to hidden layer
       for i in range(1, len(hidden_layers_neurons)):
        self.weight.append(np.random.rand(hidden_layers_neurons[i], hidden_layers_neurons[i-1]))
        self.bias.append(np.random.rand(hidden_layers_neurons[i], bias_input))


       # Hidden layer to output layer
       self.weight.append(np.random.rand(output_layer_neuron,hidden_layers_neurons[-1]))
       self.bias.append(np.random.rand(output_layer_neuron,bias_input))

   def forward_pass(self, input):
        # Make input a 2D array
        input = np.array(input).reshape(-1,1)

       # using Z = Wx + b
        z_hidden=[]
        a_hidden=[]
        for i in range(len(self.weight)):
            if i == 0:
                z = np.dot(self.weight[i], input) + self.bias[i]
            else:
                z = np.dot(self.weight[i], a_hidden[i-1]) + self.bias[i]
            z_hidden.append(z)
            a_hidden.append(self.activation_function(z))

        return a_hidden[-1]

In [41]:
# Test the Multilayer_perceptron class
def test_multilayer_perceptron():
    # Define test parameters
    input_neurons = 8
    hidden_layers = [10, 8, 6]
    output_neurons = 1
    activation_func = RELU  # Using the RELU function defined earlier

    # Create an instance of Multilayer_perceptron
    mlp = Multilayer_perceptron(hidden_layers, activation_func, input_neurons, output_neurons)

    # Check if the network structure is correct
    assert len(mlp.weight) == len(hidden_layers) + 1, "Incorrect number of weight matrices"
    assert len(mlp.bias) == len(hidden_layers) + 1, "Incorrect number of bias vectors"

    # Check dimensions of weight matrices and bias vectors
    assert mlp.weight[0].shape == (hidden_layers[0], input_neurons), "Incorrect shape of first weight matrix"
    for i in range(1, len(hidden_layers)):
        assert mlp.weight[i].shape == (hidden_layers[i], hidden_layers[i-1]), f"Incorrect shape of weight matrix {i}"
    assert mlp.weight[-1].shape == (output_neurons, hidden_layers[-1]), "Incorrect shape of output weight matrix"

    for i, neurons in enumerate(hidden_layers):
        assert mlp.bias[i].shape == (neurons, 1), f"Incorrect shape of bias vector {i}"
    assert mlp.bias[-1].shape == (output_neurons, 1), "Incorrect shape of output bias vector"

    # Test forward pass
    input_data = np.random.rand(input_neurons, 1)  # Create random input data
    output = mlp.forward_pass(input_data)

    assert output.shape == (output_neurons, 1), "Incorrect output shape"

    print("All tests passed!")

# Run the test
test_multilayer_perceptron()


All tests passed!


**Particle Swarm Optimization**

In [1]:
class Particle_Swarm_Optimization:
    def __init__(self, swarm_size, dimensions, fitness_function, alpha, beta, gamma, delta, epsilon):
        self.swarm_size = swarm_size
        self.dimensions = dimensions
        self.fitness_function = fitness_function
        self.particles_fittest = []
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.delta = delta
        self.epsilon = epsilon



    def initiailize_swarm(self):
        # Initialize the particles with random positions
        self.particles = np.random.rand(self.swarm_size, self.dimensions)
        self.velocities = np.zeros((self.swarm_size, self.dimensions))

        # Initialize the personal best positions as the particles' initial positions
        self.personal_best_positions = self.particles.copy()


        pass

    
    def assess_fitness(self, particle):
        pass

    
    def update_velocity(self, particle):
        pass

    def update_position(self, particle):
        pass


    def optimize(self, max_iterations):

        pass
        


IndentationError: expected an indented block after function definition on line 6 (31360978.py, line 9)